# Import Libraries

In [1]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import matplotlib.pyplot as plt
import random
import copy
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score, accuracy_score
from collections import defaultdict

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.transforms.functional as TF
#from efficientnet_pytorch import model as enet

import cv2
import timm

from tqdm import tqdm
from tqdm.notebook import tqdm

import albumentations as A
from albumentations import (
    Compose, OneOf, Normalize, CenterCrop, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, RandomRotate90, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, HueSaturationValue, CoarseDropout
    )
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

import logging
import time
from contextlib import contextmanager

# Config

In [2]:
class CFG:
    DATA_PATH = '../input/plant-pathology-2021-fgvc8/'
    TEST_PATH = DATA_PATH + 'test_images/'
    TRAIN_PATH = DATA_PATH + 'train_images/'
    TRAIN_CSV_PATH = DATA_PATH + 'train.csv'
    SUB_CSV_PATH = DATA_PATH + 'sample_submission.csv'
    
    MODEL_ARCH ='vgg11'

    IMG_SIZE = 224
    EPOCH = 10
    BATCH_SIZE = 32
    TEST_SIZE = 0.2
    RANDOM_STATE = 1234
    LR = 1e-4
    N_CLASS = 12
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #LABELS = ['healthy','complex','rust','frog_eye_leaf_spot','powdery_mildew','scab']
    
    LABELS_DICT = {
    0: 'complex', 
    1: 'frog_eye_leaf_spot',
    2: 'frog_eye_leaf_spot complex',
    3: 'healthy',
    4: 'powdery_mildew', 
    5: 'powdery_mildew complex',
    6: 'rust',
    7: 'rust complex',
    8: 'rust frog_eye_leaf_spot',
    9: 'scab',
    10: 'scab frog_eye_leaf_spot',
    11: 'scab frog_eye_leaf_spot complex'
    }

In [3]:
train_df = pd.read_csv(CFG.TRAIN_CSV_PATH)
sub_df = pd.read_csv(CFG.SUB_CSV_PATH)
sub = pd.read_csv(CFG.SUB_CSV_PATH)

# Seed

In [4]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

@contextmanager
def timer(name, logger=None, level=logging.DEBUG):
    print_ = print if logger is None else lambda msg: logger.log(level, msg)
    t0 = time.time()
    print_(f'[{name}] start')
    yield
    print_(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
def to_numpy(tensor):
    """Auxiliary function to convert tensors into numpy arrays
    """
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()


# Dataset

In [6]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.image_names = df['image'].values
        self.labels = df['labels'].values
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        label = self.labels[idx]
        image_path = CFG.TEST_PATH + image_name
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

# Transform

In [7]:
def get_transforms(data):
    
    if data == 'train':
        return Compose([
            A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
            #A.RandomResizedCrop(CFG.IMG_SIZE, CFG.IMG_SIZE),
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.RandomBrightnessContrast(p=0.5),

            A.Normalize(),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
            Normalize(),
            ToTensorV2(),
        ])

In [8]:
test_dataset = TestDataset(sub_df, transform = get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)

In [9]:
test_dataset[0]

tensor([[[ 1.2214,  1.1358,  1.0673,  ..., -0.7479, -1.1932, -1.1247],
         [ 1.2557,  1.2214,  1.1872,  ..., -0.2513, -0.7137, -0.5082],
         [ 1.3242,  1.3070,  1.2557,  ...,  0.3994,  0.2111,  0.1254],
         ...,
         [-1.5699, -1.5185, -1.4672,  ...,  1.1015,  1.1187,  1.3242],
         [-1.5699, -1.5699, -1.4843,  ...,  1.1529,  1.0673,  1.3070],
         [-1.5870, -1.5357, -1.5870,  ...,  0.9646,  1.0502,  1.3413]],

        [[ 1.6057,  1.6057,  1.5882,  ..., -0.2150, -0.6702, -0.6352],
         [ 1.6933,  1.6408,  1.6583,  ...,  0.2227, -0.1800, -0.1450],
         [ 1.7108,  1.7458,  1.6758,  ...,  0.7829,  0.5728,  0.5028],
         ...,
         [-1.0728, -1.0203, -1.0203,  ...,  0.9580,  1.0280,  1.2031],
         [-1.0378, -1.0378, -1.0728,  ...,  0.9755,  0.9580,  1.2206],
         [-1.0903, -1.0378, -1.1779,  ...,  0.8004,  0.9055,  1.1856]],

        [[ 1.6465,  1.6117,  1.5420,  ..., -0.6715, -1.0201, -1.0898],
         [ 1.7337,  1.7337,  1.6291,  ..., -0

# Create model

In [10]:
class VGG(nn.Module):
    """
        Base class for all neural network modules.
        Your models should also subclass this class.
        Modules can also contain other Modules, allowing to nest them in a tree structure.
        You can assign the submodules as regular attributes:
    """
    def __init__(self):
        super(VGG, self).__init__()
        self.vgg = timm.create_model(model_name = CFG.MODEL_ARCH,pretrained = False)
        in_features = self.vgg.head.fc.in_features
        self.vgg.head.fc = nn.Linear(in_features, CFG.N_CLASS)#(input_size,output_size)
        
    def forward(self, x):
        x = self.vgg(x)
        return x
    
model = VGG()

model.load_state_dict(torch.load("../input/my-vgg/vgg_model.pt"))

<All keys matched successfully>

# Prediction

In [11]:
#model.cuda()
model.eval()

predictions = []
for inputs in test_loader:
    inputs = inputs.to(CFG.DEVICE)
    with torch.no_grad():
        model = model.to(CFG.DEVICE)
        outputs = model(inputs)
        preds = outputs.argmax(1).detach().cpu().numpy()
        predictions.append(preds)
outputs,preds

(tensor([[  3.7040,  -3.9644,  -2.3690,  -4.3378,  -3.7156,  -3.4368,  -2.8898,
           -5.1783,  -6.3375,   5.9146,   2.2965,  -0.4621],
         [  6.1834,  -3.1023,   3.6206, -11.6957,  -8.3832,  -4.5229,  -6.4969,
           -3.7073,  -4.7087,  -1.6921,   5.7045,   4.3569],
         [  2.6191,   3.9252,   1.3964,  -5.4317,  -5.8872,  -5.8168,  -1.3504,
           -3.5533,  -1.2521,  -3.0800,   2.2690,  -0.4622]], device='cuda:0'),
 array([9, 0, 1]))

# Submission

In [12]:
sub['labels'] = np.concatenate(predictions)
sub = sub.replace({"labels": CFG.LABELS_DICT})
sub.to_csv('submission.csv', index=False)
sub.head()

,image,labels
0,85f8cb619c66b863.jpg,scab
1,ad8770db05586b59.jpg,complex
2,c7b03e718489f3ca.jpg,frog_eye_leaf_spot
